## Sonu Giri
> contact: sonugiri1043@gmail.com

- DataSet: Tiny ImageNet
- Model: ResNet-50 ( Without Fully Connected layer )

Tiny ImageNet dataset consists of 200 categories and each category has 500 of 64x64 size images in training set.

## Load Tiny ImageNet DataSet

In [0]:
from google.colab import drive

drive.mount( '/content/drive', force_remount=True )

Mounted at /content/drive


In [0]:
! git clone https://github.com/seshuad/IMagenet
! ls 'IMagenet/tiny-imagenet-200/'

fatal: destination path 'IMagenet' already exists and is not an empty directory.
test  train  val  wnids.txt  words.txt


## Extract test and training data from Tiny ImageNet DataSet

In [0]:
import time
import scipy.ndimage as nd
import numpy as np

path = 'IMagenet/tiny-imagenet-200/'

def get_id_dictionary():
    id_dict = {}
    for i, line in enumerate(open( path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i
    return id_dict
  
def get_class_to_id_dict():
    id_dict = get_id_dictionary()
    all_classes = {}
    result = {}
    for i, line in enumerate(open( path + 'words.txt', 'r')):
        n_id, word = line.split('\t')[:2]
        all_classes[n_id] = word
    for key, value in id_dict.items():
        result[value] = (key, all_classes[key])      
    return result

def get_data(id_dict):
    print('starting loading data')
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    t = time.time()
    for key, value in id_dict.items():
        train_data += [nd.imread( path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), mode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open( path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(nd.imread( path + 'val/images/{}'.format(img_name) ,mode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    print('finished loading data, in {} seconds'.format(time.time() - t))
    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)
  
train_data, train_labels, test_data, test_labels = get_data(get_id_dictionary())

print( "train data shape: ",  train_data.shape )
print( "train label shape: ", train_labels.shape )
print( "test data shape: ",   test_data.shape )
print( "test_labels.shape: ", test_labels.shape )

starting loading data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


finished loading data, in 57.27811360359192 seconds
train data shape:  (100000, 64, 64, 3)
train label shape:  (100000, 200)
test data shape:  (10000, 64, 64, 3)
test_labels.shape:  (10000, 200)


## Shuffle training data

In [0]:
def shuffle_data(train_data, train_labels ):
    size = len(train_data)
    train_idx = np.arange(size)
    np.random.shuffle(train_idx)

    return train_data[train_idx], train_labels[train_idx]
  
train_data, train_labels = shuffle_data(train_data, train_labels)

##Build Resnet Model

In [0]:
import six
from keras.models import Model
from keras.layers import ( Input, Activation, Dense, Flatten )
from keras.layers.convolutional import ( Conv2D, MaxPooling2D, AveragePooling2D )
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np

def _bn_relu(input):
    """Helper to build a BN -> relu block"""
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)

def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block"""
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f

def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f

def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum" """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    """Builds a residual block with repeating bottleneck blocks."""
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides,
                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input
    return f


def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                           strides=init_strides,
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                  strides=init_strides)(input)

        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)

    return f


def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4 """
    def f(input):

        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                              strides=init_strides,
                              padding="same",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                     strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1,1))(conv_3_3)
        return _shortcut(input, residual)

    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier


class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras 'Model'.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)
 
        out = Conv2D(filters=num_outputs, kernel_size=(1, 1),
                              strides=(1,1),
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(1e-4),
                              activation="softmax")(pool2)
        flatten1 = Flatten()(out)
        model = Model(inputs=input, outputs=flatten1)
        return model

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

Using TensorFlow backend.


In [0]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet50_tiny_ImageNet.csv')

batch_size = 500
nb_classes = 200
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 64, 64
# The images are RGB
img_channels = 3

# The data, shuffled and split between train and test sets:
X_train = train_data
Y_train = train_labels
X_test = test_data
Y_test = test_labels

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

model = ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [0]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
          featurewise_center=False,           # set input mean to 0 over the dataset
          samplewise_center=False,            # set each sample mean to 0
          featurewise_std_normalization=False,# divide inputs by std of the dataset
          samplewise_std_normalization=False, # divide each input by its std
          zca_whitening=False,                # apply ZCA whitening
          rotation_range=0,                   # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,              # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,             # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,               # randomly flip images
          vertical_flip=False )               # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit( X_train )

Using real-time data augmentation.


In [0]:
nb_epoch = 10 # epoch 1-10
# Fit the model on the batches generated by datagen.flow().
model.fit_generator( datagen.flow(X_train, Y_train, batch_size=batch_size),
                     steps_per_epoch=X_train.shape[0] // batch_size,
                     validation_data=(X_test, Y_test),
                     epochs=nb_epoch, verbose=1, max_q_size=100,
                     callbacks=[lr_reducer, early_stopper, csv_logger] )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  import sys


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
200/200 [==============================] - 313s 2s/step - loss: 7.8622 - acc: 0.1091 - val_loss: 6.5234 - val_acc: 0.1111
Epoch 2/10
200/200 [==============================] - 284s 1s/step - loss: 5.4329 - acc: 0.2009 - val_loss: 5.2347 - val_acc: 0.1820
Epoch 3/10
200/200 [==============================] - 282s 1s/step - loss: 4.6477 - acc: 0.2485 - val_loss: 4.6894 - val_acc: 0.2170
Epoch 4/10
200/200 [==============================] - 282s 1s/step - loss: 4.2260 - acc: 0.2819 - val_loss: 4.4114 - val_acc: 0.2402
Epoch 5/10
200/200 [==============================] - 281s 1s/step - loss: 3.9444 - acc: 0.3107 - val_loss: 4.2279 - val_acc: 0.2517
Epoch 6/10
200/200 [==============================] - 281s 1s/step - loss: 3.7360 - acc: 0.3331 - val_loss: 4.0793 - val_acc: 0.2785
Epoch 7/10
200/200 [==============================] - 283s 1s/step - loss: 3

In [0]:
epochs_passed = 10
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )

In [0]:
from keras.models import load_model

# Returns a compiled model identical to the previous one
model = load_model( 'custom_resent50_model.h5')
model.load_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 10 # epoch 11-20
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/10
200/200 [==============================] - 311s 2s/step - loss: 3.2031 - acc: 0.4109 - val_loss: 3.6135 - val_acc: 0.3344
Epoch 2/10
200/200 [==============================] - 284s 1s/step - loss: 3.1008 - acc: 0.4286 - val_loss: 3.5681 - val_acc: 0.3448
Epoch 3/10
200/200 [==============================] - 283s 1s/step - loss: 3.0402 - acc: 0.4400 - val_loss: 3.5631 - val_acc: 0.3517
Epoch 4/10
200/200 [==============================] - 286s 1s/step - loss: 2.9875 - acc: 0.4492 - val_loss: 3.5427 - val_acc: 0.3522
Epoch 5/10
200/200 [==============================] - 284s 1s/step - loss: 2.9247 - acc: 0.4614 - val_loss: 3.5898 - val_acc: 0.3516
Epoch 6/10
200/200 [==============================] - 283s 1s/step - loss: 2.8781 - acc: 0.4735 - val_loss: 3.6709 - val_acc: 0.3428
Epoch 7/10
200/200 [==============================] - 283s 1s/step - loss: 2.8397 - acc: 0.4792 - val_loss: 3.3663 - val_acc: 0.3853
Epoch 8/10
200/200 [==============================] - 282s 1s/step - 

In [0]:
epochs_passed = 20
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )

In [0]:
from keras.models import load_model
epochs_passed = 20

# Returns a compiled model identical to the previous one
model = load_model( 'custom_resent50_model.h5')
model.load_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 10 # epoch 21-30
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/10
200/200 [==============================] - 313s 2s/step - loss: 2.7222 - acc: 0.5083 - val_loss: 3.4446 - val_acc: 0.3883
Epoch 2/10
200/200 [==============================] - 280s 1s/step - loss: 2.6477 - acc: 0.5233 - val_loss: 3.5928 - val_acc: 0.3657
Epoch 3/10
200/200 [==============================] - 281s 1s/step - loss: 2.6144 - acc: 0.5319 - val_loss: 3.4228 - val_acc: 0.3854
Epoch 4/10
200/200 [==============================] - 278s 1s/step - loss: 2.5891 - acc: 0.5370 - val_loss: 3.4009 - val_acc: 0.4019
Epoch 5/10
200/200 [==============================] - 280s 1s/step - loss: 2.5640 - acc: 0.5431 - val_loss: 3.4203 - val_acc: 0.3932
Epoch 6/10
200/200 [==============================] - 281s 1s/step - loss: 2.5398 - acc: 0.5482 - val_loss: 3.5287 - val_acc: 0.3887
Epoch 7/10
200/200 [==============================] - 280s 1s/step - loss: 2.5110 - acc: 0.5564 - val_loss: 3.5288 - val_acc: 0.3833
Epoch 8/10
200/200 [==============================] - 280s 1s/step - 

In [0]:
epochs_passed = 30
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )

In [0]:
from keras.models import load_model
epochs_passed = 30

# Returns a compiled model identical to the previous one
model = load_model( 'custom_resent50_model.h5')
model.load_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

nb_epoch = 20 # epoch 31-50
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=20, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  # This is added back by InteractiveShellApp.init_path()


Epoch 1/20
200/200 [==============================] - 312s 2s/step - loss: 2.4577 - acc: 0.5752 - val_loss: 3.3310 - val_acc: 0.4151
Epoch 2/20
200/200 [==============================] - 280s 1s/step - loss: 2.3981 - acc: 0.5879 - val_loss: 3.4938 - val_acc: 0.4039
Epoch 3/20
200/200 [==============================] - 278s 1s/step - loss: 2.3730 - acc: 0.5941 - val_loss: 3.5969 - val_acc: 0.3922
Epoch 4/20
200/200 [==============================] - 278s 1s/step - loss: 2.3490 - acc: 0.6011 - val_loss: 3.6004 - val_acc: 0.3967
Epoch 5/20
200/200 [==============================] - 280s 1s/step - loss: 2.3398 - acc: 0.6040 - val_loss: 3.4154 - val_acc: 0.4206
Epoch 6/20
200/200 [==============================] - 279s 1s/step - loss: 2.3191 - acc: 0.6106 - val_loss: 3.3694 - val_acc: 0.4399
Epoch 7/20
200/200 [==============================] - 280s 1s/step - loss: 1.9283 - acc: 0.7093 - val_loss: 2.9221 - val_acc: 0.5035
Epoch 8/20
200/200 [==============================] - 277s 1s/step - 

In [0]:
epochs_passed = 47
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )
print("Model and weights after %s epochs have been saved" % epochs_passed )

Model and weights after 47 epochs have been saved


In [0]:
nb_epoch = 13 # epoch 47-60
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    validation_data=(X_test, Y_test),
                    epochs=nb_epoch, verbose=1, max_q_size=100,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=200, validation_data=(array([[[..., epochs=13, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  import sys


Epoch 1/13
200/200 [==============================] - 232s 1s/step - loss: 1.1088 - acc: 0.9068 - val_loss: 2.9980 - val_acc: 0.5161
Epoch 2/13
200/200 [==============================] - 232s 1s/step - loss: 1.0894 - acc: 0.9118 - val_loss: 3.0060 - val_acc: 0.5143
Epoch 3/13
200/200 [==============================] - 233s 1s/step - loss: 1.0760 - acc: 0.9154 - val_loss: 3.0285 - val_acc: 0.5154
Epoch 4/13
200/200 [==============================] - 233s 1s/step - loss: 1.0610 - acc: 0.9191 - val_loss: 3.0449 - val_acc: 0.5137
Epoch 5/13
200/200 [==============================] - 233s 1s/step - loss: 1.0486 - acc: 0.9216 - val_loss: 3.0565 - val_acc: 0.5128
Epoch 6/13
200/200 [==============================] - 233s 1s/step - loss: 1.0363 - acc: 0.9252 - val_loss: 3.0628 - val_acc: 0.5115
Epoch 7/13
200/200 [==============================] - 233s 1s/step - loss: 1.0210 - acc: 0.9302 - val_loss: 3.0554 - val_acc: 0.5146
Epoch 8/13
200/200 [==============================] - 234s 1s/step - 

In [0]:
epochs_passed = 60
model.save_weights( 'model_weights_Img_aug_after_%s_epoch.h5' % epochs_passed )
model.save( 'custom_resent50_model.h5' )
print("Model and weights after %s epochs have been saved" % epochs_passed )

Model and weights after 60 epochs have been saved
